# Suspension Velocity Analysis

This notebook analyzes suspension velocity distributions from shock pot displacement data. Understanding how your dampers work across different velocity ranges is crucial for proper setup.

## Interpretation Guide

| Pattern | Possible Cause | Potential Action |
|---------|----------------|------------------|
| High friction % | Damper friction, insufficient low-speed valving | Check damper seals, consider softer low-speed settings |
| Positive skew (more bump) | Nose dive, roll stiffness imbalance | Increase compression damping, check spring rates |
| Negative skew (more rebound) | Rear squat, extension issues | Increase rebound damping, check spring preload |
| High curb % | Aggressive kerb use, bottoming | Add bump stops, increase spring rate |
| Left/Right asymmetry | Weight distribution, alignment | Check corner weights, alignment, damper settings |
| Front/Rear asymmetry | Pitch balance issues | Adjust front/rear damping or spring ratio |

## What You'll Find Here

- **Velocity Histograms**: Distribution of wheel velocity for each corner (FL, FR, RL, RR)
- **Velocity Range Shading**: Visual indication of damper operating ranges
  - **Gray (Friction)**: < 5 mm/s - Static friction zone, damper may not move
  - **Light Blue (Slow)**: 5-25 mm/s - Low speed damping, affects body control
  - **Light Green (Fast)**: 25-200 mm/s - High speed damping, affects bump absorption
  - **Light Coral (Curb)**: > 200 mm/s - Curb/kerb strikes, bottoming
- **Statistics Tables**: Skew, symmetry, and corner comparisons

## Using Your Own Data

1. **Run the first cell** below to install packages and display the upload widget
2. **Click "Choose File"** to select your `.xrk` or `.xrz` file
3. **Configure channel names** if your shock pot channels have different names
4. **Run all remaining cells** to analyze your data

## Requirements

- Shock pot displacement channels (e.g., `LF_Shock_Pot`, `RF_Shock_Pot`, etc.)

**Note:** This notebook works in both JupyterLite (browser) and standard JupyterLab environments.

In [1]:
# Install required packages (needed for JupyterLite, skipped in regular JupyterLab if already installed)
%pip install -q pandas plotly libxrk motorsports-data-notebook jinja2 ipywidgets

# Import helper functions
from motorsports_data_notebook.suspension import (
    MotionRatios,
    VelocityRanges,
    SUSPENSION_CHANNEL_NAMES,
    analyze_suspension_velocity,
    format_suspension_stats_table,
    format_symmetry_table,
    format_comparison_table,
)
from motorsports_data_notebook.visualization import (
    format_lap_time,
    plot_suspension_velocity_histogram,
    show_fig,
)
from motorsports_data_notebook.widgets import SessionPicker

# Session picker - upload your own file and select a lap to analyze
session = SessionPicker(default_file="CMD_Inferno 86_Fuji GP Sh_Generic testing_a_2248.xrz")
session.display()

Note: you may need to restart the kernel to use updated packages.


division: scan=0.109979, gps=0.027013, group/ch=0.036435 more


/home/runner/.cache/pypoetry/virtualenvs/motorsports-data-notebook-lSQWGACw-py3.12/lib/python3.12/site-packages/libxrk/gps.py:343: RuntimeWarning: invalid value encountered in divide
  t = np.maximum(-np.sum(SN * O, axis=1) / np.sum(SN * D, axis=1), 0)
/home/runner/.cache/pypoetry/virtualenvs/motorsports-data-notebook-lSQWGACw-py3.12/lib/python3.12/site-packages/libxrk/gps.py:356: RuntimeWarning: divide by zero encountered in divide
  t = np.maximum(-np.sum(SN * O, axis=1) / np.sum(SN * D, axis=1), 0)
/home/runner/.cache/pypoetry/virtualenvs/motorsports-data-notebook-lSQWGACw-py3.12/lib/python3.12/site-packages/libxrk/gps.py:357: RuntimeWarning: invalid value encountered in multiply
  dist = np.sum(np.square(O + t.reshape((len(t), 1)) * D), axis=1)


In [2]:
# Get laps as pandas DataFrame for display
laps = session.get_laps()

In [3]:
# Display lap times table
laps.style.format({"lap_time": format_lap_time})  # type: ignore[dict-item]

,num,start_time,end_time,lap_time
0,0,0,150454,2:30.454
1,1,150454,279602,2:09.148
2,2,279602,406240,2:06.638
3,3,406240,532797,2:06.557
4,4,532797,659282,2:06.485
5,5,659282,787773,2:08.491
6,6,787773,913776,2:06.003
7,7,913776,1041397,2:07.621
8,8,1041397,1168322,2:06.925
9,9,1168322,1294676,2:06.354


## Configuration

Configure your shock pot channel names and motion ratios below. The defaults are set for a Toyota 86 ZN6 with standard AIM channel naming.

In [4]:
# Shock pot channel names - modify to match your data logger setup
channel_names = {
    "shock_fl": "LF_Shock_Pot",
    "shock_fr": "RF_Shock_Pot",
    "shock_rl": "LR_Shock_Pot",
    "shock_rr": "RR_Shock_Pot",
}

# Motion ratios: wheel_velocity = shock_velocity / motion_ratio
# Default: Toyota 86 ZN6 (front MacPherson: 0.997, rear multi-link: 0.768)
motion_ratios = MotionRatios(
    front_left=0.997,
    front_right=0.997,
    rear_left=0.768,
    rear_right=0.768,
)

# Velocity range thresholds (mm/s) - defines the shading regions
velocity_ranges = VelocityRanges(
    friction=5.0,  # Below: static friction zone
    slow=25.0,  # Below: low speed damping
    fast=200.0,  # Above: curb/high speed
)

## Analyze Best Lap

In [5]:
# Get the selected lap
selected_lap = session.get_selected_lap()
log = session.get_log()
lap_num = int(selected_lap["num"])
print(f"Analyzing lap {lap_num} - {format_lap_time(selected_lap['lap_time'])}")

# Filter log to selected lap using libxrk 0.5.0 methods
lap_log = log.filter_by_lap(lap_num)

# Analyze suspension velocity (receives pre-filtered log)
result = analyze_suspension_velocity(
    lap_log,
    channel_names=channel_names,
    motion_ratios=motion_ratios,
    velocity_ranges=velocity_ranges,
    smoothing_window=5,  # Rolling average to reduce noise
    bin_size=10.0,  # Histogram bin size in mm/s
)

print("Analysis complete!")

Analyzing lap 14 - 2:05.056
Analysis complete!


## Velocity Histogram Visualization

This 4-quadrant plot shows the velocity distribution for each corner:
- **Blue bars**: Bump (compression) - positive velocity
- **Red bars**: Rebound (extension) - negative velocity
- **Background shading**: Velocity range zones

In [6]:
# Plot velocity histograms
fig = plot_suspension_velocity_histogram(
    result,
    title=f"Suspension Velocity Distribution - Lap {int(selected_lap['num'])} ({format_lap_time(selected_lap['lap_time'])})",
)
show_fig(fig)

## Statistics Tables

### Per-Corner Statistics

Key metrics for each corner:
- **Skew**: Distribution asymmetry (0 = symmetric, + = more bump, - = more rebound)
- **Kurtosis**: Distribution "tailedness" (0 = normal, + = heavy tails)
- **Mean/Std**: Average velocity and spread
- **Range %**: Percentage of time spent in each velocity range

In [7]:
# Per-corner statistics
stats_df = format_suspension_stats_table(result)
stats_df.style.format(
    {
        "Skew": "{:.2f}",
        "Kurtosis": "{:.2f}",
        "Mean (mm/s)": "{:.1f}",
        "Std (mm/s)": "{:.1f}",
        "Zero Bin %": "{:.1f}",
        "Friction %": "{:.1f}",
        "Slow Bump %": "{:.1f}",
        "Slow Rebound %": "{:.1f}",
        "Fast Bump %": "{:.1f}",
        "Fast Rebound %": "{:.1f}",
        "Curb %": "{:.1f}",
    }
)

,Corner,Skew,Kurtosis,Mean (mm/s),Std (mm/s),Zero Bin %,Friction %,Slow Bump %,Slow Rebound %,Fast Bump %,Fast Rebound %,Curb %
0,FL,0.52,8.80,0.2,79.4,47.3,47.3,8.4,8.3,14.2,17.5,4.3
1,FR,0.48,13.21,0.4,73.5,48.9,48.9,9.2,11.3,12.7,14.3,3.7
2,RL,1.83,18.97,0.5,97.5,42.5,42.5,7.0,8.0,16.6,20.6,5.4
3,RR,-0.09,9.00,0.3,104.2,42.4,42.4,6.9,8.9,17.5,17.1,7.2


### Bump vs Rebound Symmetry

Compare bump (compression) vs rebound (extension) for each corner:
- **Bump/Total %**: Percentage of non-friction time spent in bump (50% = symmetric)

In [8]:
# Bump vs rebound symmetry
symmetry_df = format_symmetry_table(result)
symmetry_df.style.format(
    {
        "Total Bump %": "{:.1f}",
        "Total Rebound %": "{:.1f}",
        "Bump/Total %": "{:.1f}",
        "Slow Bump %": "{:.1f}",
        "Slow Rebound %": "{:.1f}",
        "Fast Bump %": "{:.1f}",
        "Fast Rebound %": "{:.1f}",
    }
)

,Corner,Total Bump %,Total Rebound %,Bump/Total %,Slow Bump %,Slow Rebound %,Fast Bump %,Fast Rebound %
0,FL,22.6,25.8,46.7,8.4,8.3,14.2,17.5
1,FR,21.9,25.5,46.2,9.2,11.3,12.7,14.3
2,RL,23.6,28.6,45.2,7.0,8.0,16.6,20.6
3,RR,24.4,26.0,48.4,6.9,8.9,17.5,17.1


### Left vs Right and Front vs Rear Comparison

Compare combined wheel groups to identify balance issues:
- **Left vs Right**: Combines (FL + RL) vs (FR + RR) - shows lateral balance
- **Front vs Rear**: Combines (FL + FR) vs (RL + RR) - shows longitudinal balance

Positive values mean more time in that range for Left/Front, negative for Right/Rear.

In [9]:
# Left/Right and Front/Rear comparison
comparison_df = format_comparison_table(result)
comparison_df.style.format(
    {
        "Zero Bin Diff": "{:+.1f}",
        "Friction Diff": "{:+.1f}",
        "Slow Bump Diff": "{:+.1f}",
        "Slow Rebound Diff": "{:+.1f}",
        "Fast Bump Diff": "{:+.1f}",
        "Fast Rebound Diff": "{:+.1f}",
        "Curb Diff": "{:+.1f}",
    }
)

,Comparison,Zero Bin Diff,Friction Diff,Slow Bump Diff,Slow Rebound Diff,Fast Bump Diff,Fast Rebound Diff,Curb Diff
0,Left vs Right,-0.8,-0.8,-0.3,-1.9,+0.3,+3.3,-0.6
1,Front vs Rear,+5.6,+5.6,+1.9,+1.3,-3.6,-2.9,-2.3
